In [ ]:
!pip install torch-scatter==2.0.2 -f https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.2-cp37-cp37m-linux_x86_64.whl size=7755966 sha256=05a577c68bd9a94c40dbc165e3da3dbec8d0a4f1e19137d02f4d7aad0ae8f116
  Stored in directory: /root/.cache/pip/wheels/68/94/3a/203b4e701bab9db29f54b9462e62de45e54b0bf63b4ea5ced6
Successfully built torch-scatter


In [ ]:
! rm -r transformers
! git clone https://github.com/huggingface/transformers.git
! cd transformers
! pip install ./transformers

rm: cannot remove 'transformers': No such file or directory
Cloning into 'transformers'...
remote: Enumerating objects: 86970, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 86970 (delta 36), reused 22 (delta 6), pack-reused 86884
Receiving objects: 100% (86970/86970), 70.10 MiB | 32.79 MiB/s, done.
Resolving deltas: 100% (62566/62566), done.
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████|

# Prepare Data

In [ ]:
import pickle
df = pickle.load(open('../../data/imdb/imdb_reviews_1000film.df','rb'))
ground_truth = pickle.load(open('../../data/imdb/imdb_GT.pkl','rb'))
review_ids = pickle.load(open('../../data/imdb/imdb_reviewIDs.pkl','rb'))
row_ids = pickle.load(open('../../data/imdb/imdb_movieIDs.pkl','rb'))
id_revs = inv_map = {v: k for k, v in review_ids.items()}

In [ ]:
import datetime
import csv

movies_dic = {}
with open('../../data/imdb/imdb_movielens.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for row in csv_reader:
        if row[12].replace('_',' ') not in movies_dic: 
            movies_dic[row[12].replace('_',' ')] = []
            
        temp = [r.replace('_',' ') for r in row[0:10]]
        
        month,year = '',''
        if len(row[10]) > 0:        
            month = datetime.date(1900, int(row[10][4::]), 1).strftime('%B')
            year = row[10][0:4]
        
        temp.append(month.lower() + ' ' + year)
        temp.append(int(float(row[14])))
        
        movies_dic[row[12].replace('_',' ')].append(temp)


In [ ]:
import pandas as pd

imdb_tables = []
header = ['id','info']

i = 0
for m in movies_dic:
  #for n in movies_dic[m]: 
    r = ' '.join([str(mm) for mm in movies_dic[m][0] if mm not in ['',' ']])
    #r = m + ' ' + r

    imdb_tables.append([i, r])
    i += 1

df = pd.DataFrame(imdb_tables,columns=header)
df.to_csv('imdb_all.csv',index=False)


In [ ]:
import os

#os.mkdir('tables')
chunks = [imdb_tables[x:x+500] for x in range(0, len(imdb_tables), 500)]
row_address = {}
for i in range(0,len(chunks)):
  df = pd.DataFrame(chunks[i],columns=header)
  df.to_csv(f'table_{i}.csv',index=False)

  for r in chunks[i]:
    row_address[r[1]] = (f'table_{i}.csv', chunks[i].index(r)   ) 

In [ ]:
header = ['question','table_file','answer_coordinates','answer_text']
table = []


for m in ground_truth:
    if len(ground_truth[m]) == 0 : continue

    for q in ground_truth[m]:
      if len(review_ids[q]) > 1500: continue


      #for t in movies_dic[m]:
      tt = ' '.join([str(mm) for mm in movies_dic[m][0] if mm not in ['',' ']])
      #tt = m + ' ' + tt

      ind = row_address[tt][1]
      row = [review_ids[q]]
      cor,ans = [f'({ind},{1})'],[tt]

      tfile = '' + row_address[tt][0]

      row.append(tfile)
    
      row.append(cor)
      row.append(ans)

      table.append(row)


df = pd.DataFrame(table,columns=header)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3)

train.to_csv('queries_train.csv',index=False)
test.to_csv('queries_test.csv',index=False)

len(train), len(test)

(510, 219)

In [ ]:
df

,question,table_file,answer_coordinates,answer_text
0,i watch this movie for minimum 5-6 times first...,table_46.csv,"[(15,1)]",[the shawshank redemption morgan freeman jeffr...
1,The Godfather is one of the most iconic films ...,table_81.csv,"[(22,1)]",[the godfather al pacino marlon brando robert ...
2,"Dark, yes, complex, ambitious. Christopher Nol...",table_1.csv,"[(25,1)]",[the dark knight christian bale heath ledger m...
3,"Confidently directed, dark, brooding, and pack...",table_1.csv,"[(25,1)]",[the dark knight christian bale heath ledger m...
4,"This if you have not already.It is violent, ac...",table_79.csv,"[(6,1)]",[pulp fiction bruce willis eric stoltz phil la...
...,...,...,...,...
724,Here is a tragedy -- a great film doomed by a ...,table_246.csv,"[(20,1)]",[duck you sucker james coburn rod steiger romo...
725,What can I say about The Jungle Book? It proba...,table_1.csv,"[(36,1)]",[the jungle book scarlett johansson bill murra...
726,There aren't many animated Disney films I don'...,table_1.csv,"[(36,1)]",[the jungle book scarlett johansson bill murra...
727,There has always been a debate regarding what ...,table_101.csv,"[(6,1)]",[from here to eternity montgomery clift donna ...


## Prepare the data 

Let's look at the first few rows of the dataset:

In [ ]:
import pandas as pd
data = pd.read_csv("queries_train.csv")
data.head()

,question,table_file,answer_coordinates,answer_text
0,"I mean, i WAS pretty high, but still.It defini...",table_280.csv,"['(17,1)']",['hamilton peter stormare mats lngbacka lena o...
1,This is just a masterpiece. It is probably the...,table_250.csv,"['(5,1)']",['the day of the jackal edward fox michael lon...
2,Key Largo is an absolutely brilliant film. Cas...,table_167.csv,"['(0,1)']",['key largo humphrey bogart edward g robinson ...
3,"Earlier to, many movies tried to capture the i...",table_558.csv,"['(30,1)']",['kai po che amrita puri amit sadh rajkummar r...
4,Kokuhaku (or Confessions) is a real winner fro...,table_430.csv,"['(5,1)']",['confessions masaki okada yoshino kimura tets...


In [ ]:
import ast

def _parse_answer_coordinates(answer_coordinate_str):
  """Parses the answer_coordinates of a question.
  Args:
    answer_coordinate_str: A string representation of a Python list of tuple
      strings.
      For example: "['(1, 4)','(1, 3)', ...]"
  """

  try:
    answer_coordinates = []
    # make a list of strings
    coords = ast.literal_eval(answer_coordinate_str)
    # parse each string as a tuple
    for row_index, column_index in sorted(
        ast.literal_eval(coord) for coord in coords):
      answer_coordinates.append((row_index, column_index))
  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_coordinate_str)
  
  return answer_coordinates


def _parse_answer_text(answer_text):
  """Populates the answer_texts field of `answer` by parsing `answer_text`.
  Args:
    answer_text: A string representation of a Python list of strings.
      For example: "[u'test', u'hello', ...]"
    answer: an Answer object.
  """
  try:
    answer = []
    for value in ast.literal_eval(answer_text):
      answer.append(value)
  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_text)

  return answer

data['answer_coordinates'] = data['answer_coordinates'].apply(lambda coords_str: _parse_answer_coordinates(coords_str))
data['answer_text'] = data['answer_text'].apply(lambda txt: _parse_answer_text(txt))

data.head(10)

,question,table_file,answer_coordinates,answer_text
0,"I mean, i WAS pretty high, but still.It defini...",table_280.csv,"[(17, 1)]",[hamilton peter stormare mats lngbacka lena ol...
1,This is just a masterpiece. It is probably the...,table_250.csv,"[(5, 1)]",[the day of the jackal edward fox michael lons...
2,Key Largo is an absolutely brilliant film. Cas...,table_167.csv,"[(0, 1)]",[key largo humphrey bogart edward g robinson l...
3,"Earlier to, many movies tried to capture the i...",table_558.csv,"[(30, 1)]",[kai po che amrita puri amit sadh rajkummar ra...
4,Kokuhaku (or Confessions) is a real winner fro...,table_430.csv,"[(5, 1)]",[confessions masaki okada yoshino kimura tetsu...
5,I was sceptical before watching this film but ...,table_50.csv,"[(22, 1)]",[joyeux noel gary lewis dany boon ian richards...
6,This was a surprisingly intense film that I'm ...,table_207.csv,"[(15, 1)]",[the experiment moritz bleibtreu christian ber...
7,How great was this movie? I fell in love with ...,table_38.csv,"[(7, 1)]",[remember the titans ryan gosling denzel washi...
8,Top 10 comedies of all time. Enough jokes and ...,table_33.csv,"[(20, 1)]",[the hangover bradley cooper rob riggle mike e...
9,"David Lean's ""Doctor Zhivago"" is a classic fil...",table_71.csv,"[(25, 1)]",[doctor zhivago julie christie klaus kinski ge...


In [ ]:
data = data.sort_values(by=['answer_coordinates'], ascending=True)
data.head(10)

,question,table_file,answer_coordinates,answer_text
52,This movie was stunning in many aspects. Visua...,table_60.csv,"[(0, 1)]",[the illusionist rufus sewell eddie marsan jak...
19,"''Kakushi-toride no san-akunin'', better well ...",table_247.csv,"[(0, 1)]",[the hidden fortress toshir mifune takashi shi...
75,"Other reviewers, aside from seeing this as the...",table_94.csv,"[(0, 1)]",[the man who shot liberty valance lee marvin w...
95,"Modern Times owes a lot to Metropolis, despite...",table_102.csv,"[(0, 1)]",[modern times paulette goddard stanley blyston...
354,I saw this with my family in a theatre. Superb...,table_79.csv,"[(0, 1)]",[my name is khan shah rukh khan jimmy shergill...
196,Well... O.K.! I'm gonna say the same things th...,table_253.csv,"[(0, 1)]",[white heat james cagney virginia mayo edmond ...
492,After reading on the BBC that this was voted t...,table_32.csv,"[(0, 1)]",[serenity adam baldwin sean maher michael hitc...
388,"At first glance, you would think this movie wo...",table_51.csv,"[(0, 1)]",[silver linings playbook jennifer lawrence rob...
252,***May contain spoilers***If you watched Befor...,table_95.csv,"[(0, 1)]",[before midnight seamus daveyfitzpatrick arian...
494,"If you saw The Prestige, prepare yourself for ...",table_60.csv,"[(0, 1)]",[the illusionist rufus sewell eddie marsan jak...


In [ ]:
import torch
from transformers import TapasTokenizer
table_csv_path = ''

# initialize the tokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base")

In [ ]:
len(data.iloc[4].question)

662

In [ ]:
table_csv_path = ''
item = data.iloc[1]

table = pd.read_csv(table_csv_path + item.table_file).astype(str) 

display(table)
print(item.question)

encoding = tokenizer(table=table, queries=item.question, answer_coordinates=item.answer_coordinates, answer_text=item.answer_text,
                     truncation=True, padding="max_length", return_tensors="pt")
encoding.keys()

In [ ]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        table = pd.read_csv(table_csv_path + item.table_file).astype(str) # TapasTokenizer expects the table data to be text only
        encoding = self.tokenizer(table=table,
                                   queries=item.question,
                                   answer_coordinates=item.answer_coordinates,
                                   answer_text=item.answer_text,
                                   truncation=True,
                                   padding="max_length",
                                   return_tensors="pt"
         )

          # use encodings of second table-question pair in the batch
        encoding = {key: val[-1] for key, val in encoding.items()}

        #encoding["float_answer"] = torch.tensor(item.float_answer)



        return encoding

    def __len__(self):
        return len(self.df)

#data = pd.read_csv('data/queries.csv')
train_dataset = TableDataset(df=data, tokenizer=tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8)

In [ ]:
train_dataset[0]["token_type_ids"].shape

torch.Size([512, 7])

In [ ]:
train_dataset[1]["input_ids"].shape

torch.Size([512])

In [ ]:
batch = next(iter(train_dataloader))

In [ ]:
batch["input_ids"].shape

torch.Size([8, 512])

In [ ]:
batch["token_type_ids"].shape

torch.Size([8, 512, 7])

Let's decode the first table-question pair:

In [ ]:
tokenizer.decode(batch["input_ids"][0])

'[CLS] hero is noteworthy on at least two counts. first, there are scenes of haunting beauty ( " duel in the yellow forest " and " turquoise autumn " to site a couple ) that, like the best of impressionist paintings, are so affecting that you will forever see the world in a slightly different way having once beheld them. secondly, the overall message of the film is a provocative one. the claim is that a degree of human casualties and suffering may be the optimal path to a better world, especially when the alternative is equally brutal chaos. this is not a popular theme. it has become much more fashionable to be anti - war in all cases. and understandably so, since variations of this logic have often been used in the past to justify atrocities. but the film provides a crisp litmus test for avoiding delusion : action must be taken with a heart void of malice and an unwavering commitment to the broadest possible ultimate outcome of good for all. can anyone live up to this standard? severa

In [ ]:
#first example should not have any prev_labels set
assert batch["token_type_ids"][0][:,3].sum() == 0

Let's decode the second table-question pair and verify some more:

In [ ]:
tokenizer.decode(batch["input_ids"][1])

"[CLS] a scorching performance from bogey makes this film a real classic, his dixon steele one of the great screen characters. in this more biting version of the plot of hitchcock's suspense / comedy suspicion, bogart is a kind and loving screenwriter with a violent streak of temper waiting to break out and a taste for a drink or two, wooing gloria grahame's pretty young actress next door. the death of a young girlfriend of his hangs over him throughout the movie, as graham at first believes him to be innocent, then later, having fallen for his charms, begins to suspect he may have had something to do with the girl's death after all, as his temper becomes more and more uncontrolled and frightening. the police circle around, making his nervous anger worse ; the relationship begins to crumble into a mess of fear, lies and misunderstanding. through all this dixon steele emerges as a great and brilliant creation, a highlight even in a career as illustrious as that of bogart, a charming and

In [ ]:
for id, prev_label in zip(batch["input_ids"][1], batch["token_type_ids"][1][:,3]):
  if id != 0:
    print(tokenizer.decode([id]), prev_label.item())

## Define the model

Here we initialize the model with a pre-trained base and randomly initialized cell selection head, and move it to the GPU (if available).

Note that the `google/tapas-base` checkpoint has (by default) an SQA configuration, so we don't need to specify any additional hyperparameters.

In [ ]:
from transformers import TapasForQuestionAnswering

model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['output_weights', 'column_output_bias', 'column_output_weights', 'output_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TapasForQuestionAnswering(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings_0): Embedding(3, 768)
      (token_type_embeddings_1): Embedding(256, 768)
      (token_type_embeddings_2): Embedding(256, 768)
      (token_type_embeddings_3): Embedding(2, 768)
      (token_type_embeddings_4): Embedding(256, 768)
      (token_type_embeddings_5): Embedding(256, 768)
      (token_type_embeddings_6): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.07, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0): TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)


## Training the model

Let's fine-tune the model in well-known PyTorch fashion:

In [ ]:
from transformers import AdamW
import time

st = time.time()

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(15):  # loop over the dataset multiple times
   print("Epoch:", epoch)
   for idx, batch in enumerate(train_dataloader):
        # get the inputs;
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["labels"].to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,
                       labels=labels)
        loss = outputs.loss
        print("Loss:", loss.item())
        loss.backward()
        optimizer.step()

Epoch: 0
Loss: 1.3265039920806885
Loss: 0.5520153045654297
Loss: 0.20379555225372314
Loss: 0.19696374237537384
Loss: 0.13729920983314514
Loss: 0.1245574802160263
Loss: 0.12099535018205643
Loss: 0.11510320007801056
Loss: 0.11508418619632721
Loss: 0.11508648842573166
Loss: 0.11552755534648895
Loss: 0.11610338091850281
Loss: 0.11270591616630554
Loss: 0.11529648303985596
Loss: 0.11626183986663818
Loss: 0.11623670160770416
Loss: 0.11624813079833984
Loss: 0.1168031245470047
Loss: 0.11669737100601196
Loss: 0.11594832688570023
Loss: 0.11562749743461609
Loss: 0.11483243107795715
Loss: 0.11452944576740265
Loss: 0.11433851718902588
Loss: 0.11450556665658951
Loss: 0.11567233502864838
Loss: 0.11503340303897858
Loss: 0.1158408373594284
Loss: 0.11694205552339554
Loss: 0.11597777903079987
Loss: 0.11490742117166519
Loss: 0.11334699392318726
Loss: 0.11324946582317352
Loss: 0.11298384517431259
Loss: 0.11273432523012161
Loss: 0.11309194564819336
Loss: 0.11281004548072815
Loss: 0.11247087270021439
Loss: 0.

In [ ]:
model.save_pretrained("imdb_tapas.mdl")

## Inference

As SQA is a bit different due to its conversational nature, we need to run every training example of the a batch one by one through the model (sequentially), overwriting the `prev_labels` token types (which were created by the tokenizer) by the answer predicted by the model. It is based on the [following code](https://github.com/google-research/tapas/blob/f458b6624b8aa75961a0ab78e9847355022940d3/tapas/experiments/prediction_utils.py#L92) from the official implementation:

In [ ]:
from transformers import TapasForQuestionAnswering

model = TapasForQuestionAnswering.from_pretrained("imdb_tapas.mdl")
device='cuda'

In [ ]:
test = pd.read_csv("queries_test.csv")

In [ ]:
from tqdm import tqdm
preds_tapas = {}

for i in tqdm(range(0,len(test)),position=0):
  item = test.iloc[i]
  queries = [item['question']]
  table = pd.read_csv(item['table_file']).astype(str)
  inputs = tokenizer(table=table, queries=queries, padding='max_length', return_tensors="pt")
  outputs = model(**inputs)
  predicted_answer_coordinates = tokenizer.convert_logits_to_predictions(
          inputs,
          outputs.logits.detach()
  )
  preds_tapas[item['question']] = [table.iloc[p[0]]['info'] for p in predicted_answer_coordinates[0][0]]
  print(predicted_answer_coordinates)

In [ ]:
########################## METRICS

def HAS_POSITIVE(actual,preds):
    for i in range(0,len(preds)):
        if preds[i] in actual:
            return 1
    return 0

def MRRR(actual,preds):
    for i in range(0,len(preds)):
        if preds[i] in actual:
            return 1/(i+1)
    return 0


def MAP_K(actual,preds):
    precision = 0
    hit = 0
    for i in range(0,len(preds)):
        if preds[i] in actual:
            hit += 1
            precision += hit/(i+1)
    return precision/len(actual)    


In [ ]:
id_reviews = {v: k for k, v in review_ids.items()}

preds_movie = {}

for movie in tqdm(movies_dic):
  for n in movies_dic[movie]: 
    r = ' '.join([str(mm) for mm in n if mm not in ['',' ']])
    
    temp = []

    for p in preds_tapas:
        rev = id_reviews[p]
        if r in preds_tapas[p]:
          temp.append(rev) 
    preds_movie[movie] = temp


100%|██████████| 42206/42206 [00:02<00:00, 14918.84it/s]


In [ ]:
id_reviews = {v: k for k, v in review_ids.items()}
for KK in [1,5,20]: 
    i = 0
    precision,recall,fs = 0,0,0
    MAP, MRR, hasP = 0,0,0

    for claim in preds_movie:
        if len(preds_movie[claim]) == 0 or claim not in ground_truth: continue
        i+=1
        preds =  preds_movie[claim] [0:KK]
        golds = [f for f in ground_truth[claim]]

        MAP += MAP_K(golds,preds)
        MRR += MRRR(preds,golds)
        hasP += HAS_POSITIVE(preds,golds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MRR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)



#################### 1 ###########################



ZeroDivisionError: ignored